# Model calibration using the trajectory

This is the module that using a predefined "trajectory" to tune 
parameters of mixed-in effect

<style>
    body {
        --vscode-font-family: "Lato"
    }
    h1, h2, h3 {
        
    }
</style>
<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script src="https://cdn.plot.ly/plotly-2.6.3.min.js"></script>

## Initialization

In [1]:
ENV["CRIMINOS_CONF"] = "confs/conf_single_nij.yaml"
ENV["CRIMINOS_ALIAS"] = "test"
Base.istextmime(::MIME"application/vnd.plotly.v1+json") = true
using Revise
using ForwardDiff
using LinearAlgebra
using Random
using Printf
using LaTeXStrings
using JuMP
using Criminos
using Plots
using Gurobi
using ProgressMeter
using ColorSchemes

using CSV, Tables, DataFrames, YAML

include("./conf.jl")
include("./tools.jl")

# yaml = YAML.load_file("ipynb/nij-simulation.0.15.yaml")
# yaml = yaml2 = YAML.load_file("ipynb/nij-simulation.0.15.yaml")
yaml = yaml2 = YAML.load_file("ipynb/nij-simulation.yaml")


# maximum number of time periods used in the estimation
Tₘ = 50

data_fixed = AgentData(yaml, Tₘ)
data_simulation = AgentData(yaml2, Tₘ; n=data_fixed.n)

ERROR: LoadError: InitError: could not load library "/Applications/copt70/lib/libcopt.dylib"
dlopen(/Applications/copt70/lib/libcopt.dylib, 0x0001): tried: '/Applications/copt70/lib/libcopt.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Applications/copt70/lib/libcopt.dylib' (no such file), '/Applications/copt70/lib/libcopt.dylib' (no such file)
Stacktrace:
  [1] _get_banner()
    @ COPT ~/.julia/packages/COPT/lA4VQ/src/COPT.jl:48
  [2] __init__()
    @ COPT ~/.julia/packages/COPT/lA4VQ/src/COPT.jl:121
  [3] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1134
  [4] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1122
  [5] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any})
    @ Base ./loading.jl:1067
  [6] _require_search_from_serialized(pkg::Base.PkgId, sourcepath::String, build_id::UInt128)
    @ Base ./loading.jl:1581
  [7] _require(pkg::Base.PkgId, e

ErrorException: Failed to precompile Criminos [dee8dd8d-dc08-48fb-8db7-8e301efd132a] to "/Users/brent/.julia/compiled/v1.10/Criminos/jl_LjUyqV".

In [2]:
data_simulation.γ

UndefVarError: UndefVarError: `data_simulation` not defined

## Start

> agent-based simulation results

In [3]:
n = data_fixed.n

UndefVarError: UndefVarError: `data_fixed` not defined

In [4]:
data_fixed.traj_x

UndefVarError: UndefVarError: `data_fixed` not defined

### Initialization and override the parameters of $\Psi$

In [5]:
data = data_fixed
include("./init.jl")
# baseline policy
for _z in vec_z
    _z.τ .= 0.3
end
vec_Ψ[1].γ = data.γ
vec_Ψ[1].λ = data.λ
# vec_Ψ[1].λ[1] = 497.26
vec_Ψ[1].Γ = diagm(data.γ)
vec_Ψ[1].Γₕ = vec_Ψ[1].M * vec_Ψ[1].Γ
vec_z[1].x .= data.traj_x[:, 1]
vec_z[1].x₋ .= data.traj_x[:, 1]
vec_z[1].y = data.traj_y[:, 1]

UndefVarError: UndefVarError: `data_fixed` not defined

In [6]:
Criminos.default_barrier_option.μ

UndefVarError: UndefVarError: `Criminos` not defined

## Fitting

In [7]:
_args = (ω∇ω, G, md, _H, _g) = fit_trajectory(data, vec_Ψ)

UndefVarError: UndefVarError: `fit_trajectory` not defined

In [8]:
_H, _g

UndefVarError: UndefVarError: `_H` not defined

## Using fitted params at fixed control $\tau$

In [9]:
Fp(vec_z) = F!(
    vec_z, vec_Ψ;
    fₜ=Criminos.decision_identity!, targs=(cₜ, cc.τₗ, cc.τₕ),
    fₘ=cc.style_mixin, margs=_args,
)
kₑ, ε, traj, bool_opt = Criminos.simulate(
    vec_z, vec_Ψ, Fp; K=K,
    metrics=metrics,
    tol=1e-7
);

UndefVarError: UndefVarError: `K` not defined

In [10]:
plot_convergence(ε, vec_z |> length)
# the equilibria
r = traj[end][1]

UndefVarError: UndefVarError: `ε` not defined

#### Visualize

In [11]:
figs_vis = plot_fitting_results(data, r; bool_pdf=false);

UndefVarError: UndefVarError: `plot_fitting_results` not defined

In [12]:
plot(figs_vis[1], figs_vis[2])

UndefVarError: UndefVarError: `figs_vis` not defined

In [13]:
plot(figs_vis[3], figs_vis[4])

UndefVarError: UndefVarError: `figs_vis` not defined

In [14]:
# savefig(fig, "nij-simulation-y.pdf")
# savefig(fig1, "nij-simulation-x.pdf")
# savefig(fig2, "nij-simulation-y-prog.pdf")
# savefig(fig3, "nij-simulation-x-prog.pdf")

## Dynamic control 

In [15]:
Fp(vec_z) = F!(
    vec_z, vec_Ψ;
    fₜ=Criminos.decision_priority_by_opt!, targs=(cₜ, cc.τₗ, cc.τₕ),
    fₘ=cc.style_mixin, margs=_args,
)
kₑ, ε, traj, bool_opt = Criminos.simulate(
    vec_z, vec_Ψ, Fp; K=K,
    metrics=metrics,
    tol=1e-7
);

UndefVarError: UndefVarError: `K` not defined

In [16]:
@info """sanity check 
$(abs.(Criminos.decision_priority!(traj[end][1], vec_Ψ[1]; args=(cₜ, cc.τₗ, cc.τₕ)) - traj[end][1].I) |> maximum)
"""

UndefVarError: UndefVarError: `traj` not defined

In [17]:
fig = generate_empty(cc.bool_use_html)
plot!(
    1:n, traj[end][1].y,
    labels=L"$\rho_T$",
    linestyle=:dot,
    color=series_color[1],
    linewidth=3,
)
plot!(
    1:n, data.traj_y_lb[:,end], fillrange=data.traj_y_ub[:,end],
    labels=L"$\bar \rho_T",
    fillalpha=0.3,
    linestyle=:dot,
    color=series_color[1],
)
fig

UndefVarError: UndefVarError: `cc` not defined

In [18]:
cc

UndefVarError: UndefVarError: `cc` not defined

In [19]:
[traj[end][1].τ r.τ]

UndefVarError: UndefVarError: `traj` not defined